[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

In [ ]:
#@title <b>Time Out Preventer (Advanced)</b>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent Google Colab from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }

  setInterval(ClickConnect,60000)
  '''))

In [ ]:
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip install -r requirements.txt
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d models/clip -o qwen_3_4b.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d models/vae -o ae.safetensors

In [ ]:
%cd /content/ComfyUI
import sys
sys.path.insert(0, '/content/ComfyUI')

import os, random, time, math
import torch
import numpy as np
from PIL import Image

assert torch.cuda.is_available(), "GPU is required for this notebook"

from nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS['UNETLoader']()
CLIPLoader = NODE_CLASS_MAPPINGS['CLIPLoader']()
VAELoader = NODE_CLASS_MAPPINGS['VAELoader']()
CLIPTextEncode = NODE_CLASS_MAPPINGS['CLIPTextEncode']()
KSampler = NODE_CLASS_MAPPINGS['KSampler']()
VAEDecode = NODE_CLASS_MAPPINGS['VAEDecode']()
EmptyLatentImage = NODE_CLASS_MAPPINGS['EmptyLatentImage']()

with torch.inference_mode():
    unet = UNETLoader.load_unet('z-image-turbo-fp8-e4m3fn.safetensors', 'fp8_e4m3fn_fast')[0]
    clip = CLIPLoader.load_clip('qwen_3_4b.safetensors', type='lumina2')[0]
    vae = VAELoader.load_vae('ae.safetensors')[0]

PRESETS = {
    # Quality Presets
    'Cinematic HQ': 'cinematic lighting, ultra realistic, 8k detail, film still, shallow depth of field, dramatic contrast, sharp focus, global illumination',
    'Portrait HQ': 'studio portrait, ultra realistic skin texture, subsurface scattering, soft rim light, high detail face, sharp focus',
    'Anime HQ': 'anime style, extremely detailed illustration, clean lineart, vibrant colors, high resolution, sharp shading',
    'Product HQ': 'product photography, ultra sharp focus, studio lighting, commercial quality, global illumination, clean background',
    'NSFW Artistic HQ': 'artistic nude, sensual pose, fine art photography, soft cinematic lighting, detailed skin texture, aesthetic composition'
    # Performance Presets (T4 GPU Optimized)
    'Fast Mode': 'high quality, detailed, sharp focus',
    'Balanced Mode': 'highly detailed, sharp focus, professional quality, vibrant colors',
    'Best Quality': 'ultra detailed, masterpiece, best quality, sharp focus, high resolution, cinematic lighting'
}

BASE_NEG = 'blurry, low quality, jpeg artifacts, bad anatomy, deformed, poorly drawn, oversaturated'
SAFE_NEG = 'explicit sexual acts, porn, extreme fetish, gore'

@torch.inference_mode()
def generate(v):
    out = '/content/ComfyUI/output'
    os.makedirs(out, exist_ok=True)

    prompt = f"{PRESETS[v['preset']]}, {v['prompt']}"
    negative = v.get('negative', BASE_NEG if v['nsfw'] else f"{BASE_NEG}, {SAFE_NEG}")

    seed = v['seed'] if v['seed'] != 0 else random.randint(0, 2**64-1)

    pos = CLIPTextEncode.encode(clip, prompt)[0]
    neg = CLIPTextEncode.encode(clip, negative)[0]

    latent = EmptyLatentImage.generate(v['width'], v['height'], batch_size=v['batch'])[0]
    samples = KSampler.sample(
        unet, seed,
        v['steps'], v['cfg'],
        v['sampler'], v['scheduler'],
        pos, neg, latent,
        denoise=v['denoise']
    )[0]

    decoded = VAEDecode.decode(vae, samples)[0]
    imgs = []

    for i in range(v['batch']):
        img_array = decoded[i].cpu().numpy()
        img_array = np.clip(img_array * 255, 0, 255).astype('uint8')
        img = Image.fromarray(img_array)
        imgs.append(img)

    return imgs

def grid(imgs, cols):
    rows = math.ceil(len(imgs)/cols)
    w, h = imgs[0].size
    g = Image.new('RGB', (cols*w, rows*h))
    for i, img in enumerate(imgs):
        g.paste(img, ((i%cols)*w, (i//cols)*h))
    return g

In [ ]:
import ipywidgets as w
from IPython.display import display, clear_output

# Performance Mode Selector (T4 GPU Optimized)
performance_mode = w.Dropdown(
    options=['Fast Mode', 'Balanced Mode', 'Best Quality'],
    value='Balanced Mode',
    description='Performance:',
    style={'description_width': '120px'}
)

# Quality Presets
preset = w.Dropdown(
    options=list(PRESETS.keys()),
    value='Balanced Mode',
    description='Style Preset:',
    style={'description_width': '120px'}
)

nsfw = w.Checkbox(value=False, description='NSFW')

# Prompts in columns
prompt = w.Textarea(
    value='a beautiful landscape, mountains and lake, golden hour',
    placeholder='Describe what you want to generate...',
    description='Prompt:',
    layout=w.Layout(width='100%', height='80px'),
    style={'description_width': '120px'}
)

negative_prompt = w.Textarea(
    value='',
    placeholder='What to avoid (optional, auto-filled based on NSFW setting)...',
    description='Negative:',
    layout=w.Layout(width='100%', height='80px'),
    style={'description_width': '120px'}
)

# Resolution settings
width = w.IntSlider(
    value=1024, min=512, max=2048, step=64,
    description='Width:',
    style={'description_width': '120px'}
)
height = w.IntSlider(
    value=1024, min=512, max=2048, step=64,
    description='Height:',
    style={'description_width': '120px'}
)

# Advanced settings (auto-configured by performance mode)
steps = w.IntSlider(
    value=20, min=6, max=40, step=1,
    description='Steps:',
    style={'description_width': '120px'}
)
cfg = w.FloatSlider(
    value=2.5, min=0.5, max=6.0, step=0.1,
    description='CFG Scale:',
    style={'description_width': '120px'}
)
denoise = w.FloatSlider(
    value=1.0, min=0.6, max=1.0, step=0.05,
    description='Denoise:',
    style={'description_width': '120px'}
)
batch = w.IntSlider(
    value=1, min=1, max=4, step=1,
    description='Batch Size:',
    style={'description_width': '120px'}
)
seed = w.IntText(
    value=0,
    description='Seed (0=Random):',
    style={'description_width': '120px'}
)

sampler = w.Dropdown(
    options=['dpmpp_2m', 'euler', 'euler_ancestral', 'dpm_fast'],
    value='dpmpp_2m',
    description='Sampler:',
    style={'description_width': '120px'}
)
scheduler = w.Dropdown(
    options=['karras', 'simple'],
    value='karras',
    description='Scheduler:',
    style={'description_width': '120px'}
)

# Output settings
use_grid = w.Checkbox(value=False, description='Grid Layout')
cols = w.IntSlider(
    value=2, min=2, max=4, step=1,
    description='Grid Columns:',
    style={'description_width': '120px'}
)

# Auto-configure settings based on performance mode
def update_performance(change):
    mode = change['new']
    if mode == 'Fast Mode':
        steps.value = 12
        cfg.value = 2.0
        sampler.value = 'dpm_fast'
        scheduler.value = 'simple'
        batch.max = 4
    elif mode == 'Balanced Mode':
        steps.value = 20
        cfg.value = 2.5
        sampler.value = 'dpmpp_2m'
        scheduler.value = 'karras'
        batch.max = 2
    else:  # Best Quality
        steps.value = 28
        cfg.value = 3.0
        sampler.value = 'dpmpp_2m'
        scheduler.value = 'karras'
        batch.max = 1

performance_mode.observe(update_performance, names='value')

# Sync preset dropdown with performance mode
def sync_preset(change):
    if change['new'] in ['Fast Mode', 'Balanced Mode', 'Best Quality']:
        performance_mode.value = change['new']

preset.observe(sync_preset, names='value')

btn = w.Button(
    description='🚀 GENERATE',
    button_style='success',
    layout=w.Layout(width='200px', height='40px')
)
out = w.Output()

def run(b):
    with out:
        clear_output(wait=True)
        
        # Use custom negative prompt or auto-fill
        custom_neg = negative_prompt.value.strip()
        if custom_neg:
            final_negative = custom_neg
        else:
            final_negative = BASE_NEG if nsfw.value else f"{BASE_NEG}, {SAFE_NEG}"
        
        v = dict(
            preset=preset.value,
            nsfw=nsfw.value,
            prompt=prompt.value,
            negative=final_negative,
            width=width.value,
            height=height.value,
            steps=steps.value,
            cfg=cfg.value,
            denoise=denoise.value,
            batch=batch.value,
            seed=seed.value,
            sampler=sampler.value,
            scheduler=scheduler.value
        )
        
        print(f"⚙️ Mode: {performance_mode.value} | Steps: {v['steps']} | CFG: {v['cfg']} | Sampler: {v['sampler']}")
        print(f"📐 Resolution: {v['width']}x{v['height']} | Batch: {v['batch']}")
        print(f"🎲 Seed: {v['seed'] if v['seed'] != 0 else 'Random'}\n")
        
        try:
            imgs = generate(v)
            if use_grid.value and len(imgs) > 1:
                display(grid(imgs, cols.value))
            else:
                for i in imgs:
                    display(i)
            print(f"\n✅ Generated {len(imgs)} image(s) successfully!")
        except Exception as e:
            print(f"❌ Error during generation: {e}")
            import traceback
            traceback.print_exc()

btn.on_click(run)

# Layout in organized sections
display(w.VBox([
    w.HTML('<h3 style="margin-bottom:10px;">🎨 Z-Image Turbo Generator (T4 Optimized)</h3>'),
    w.HTML('<hr style="margin:5px 0;">'),
    w.HTML('<b>Performance & Style</b>'),
    w.HBox([performance_mode, preset, nsfw]),
    w.HTML('<b>Prompts</b>'),
    prompt,
    negative_prompt,
    w.HTML('<b>Image Settings</b>'),
    w.HBox([width, height]),
    w.HBox([batch, seed]),
    w.HTML('<b>Advanced Settings</b>'),
    w.HBox([steps, cfg]),
    w.HBox([sampler, scheduler]),
    w.HBox([denoise, use_grid, cols]),
    w.HTML('<hr style="margin:10px 0;">'),
    btn,
    out
], layout=w.Layout(padding='15px'))
)
